# 爬蟲抓取證交所各股資料

In [1]:
import requests
import socket
import time
import csv, json

In [2]:
def twodigit(n):  #將數值轉為二位數字串
    if(n < 10):
        retstr = '0' + str(n)
    else:
        retstr = str(n)
    return retstr

def transform_date(date):   #民國轉西元
        y, m, d = date.split('/')
        return str(int(y)+1911) + '/' + m  + '/' + d

def transform_data(jdata):   #去除，號
    for m in range(len(jdata)):
        jdata[m][0] = transform_date(jdata[m][0])
        for n in range(len(jdata[0])):
            jdata[m][n] = jdata[m][n].replace(",", "")
    return jdata

In [3]:
def get_data(symbol):
    with open(f'./data/{symbol}.csv', 'w+', newline = '') as csvfile:
        writer = csv.writer(csvfile)
        
        writer.writerow(['date', 'trade_volume', 'trade_value', 'opening_price', 'highest_price', 'lowest_price', 'closing_price', 'change', 'transaction'])
        
        for year in range(2011, 2021):
            for month in range(1, 13):
                
                url = f'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={year}{twodigit(month)}01&stockNo={symbol}&_=1521363562193'
                while True:
                    try:
                        res = requests.get(url)   #回傳為json資料
                        break
                    except socket.error as error:
                        print(error)
                        time.sleep(120)
                        continue
                                
                jres = json.loads(res.text)  #json解析
                if 'data' in jres:
                    jdata = jres['data']
                    jdata = transform_data(jdata)                   
                    writer.writerows(jdata)
                else:
                    print(f"{year}{twodigit(month)} error")
                    break
                time.sleep(5)
            else: continue
            break

In [4]:
if __name__ == '__main__':
    symbol = 2412
    get_data(symbol)